In [2]:
import torch
import os
from PIL import Image
import numpy as np


def load_video_to_tensor(directory_path):
    # 1. Pobierz listę plików i posortuj je (ważne, aby zachować kolejność klatek!)
    files = sorted([f for f in os.listdir(directory_path) if f.endswith((".png", ".jpg", ".jpeg"))])

    frames = []

    for file in files:
        img_path = os.path.join(directory_path, file)
        img = Image.open(img_path).convert("RGB")  # convert zapewnia 3 kanały
        # Konwersja do tensora uint8 [0, 255]
        img_tensor = torch.from_numpy(np.array(img)).permute(2, 0, 1)
        frames.append(img_tensor)

    # 2. Łączenie listy tensorów w jeden tensor 4D
    # Wynikowy shape: [T, C, H, W]
    video_tensor = torch.stack(frames)

    return video_tensor


# Użycie:
# tensor = load_video_to_tensor('sciezka/do/klatek')
# print(tensor.shape) # np. torch.Size([30, 3, 720, 1280])


In [4]:
from torchcodec.encoders import VideoEncoder
import os

os.makedirs("../../videolq_videos", exist_ok=True)

for i in os.listdir("../../Input"):
    tensor = load_video_to_tensor(f"../../Input/{i}")
    encoder = VideoEncoder(frames=tensor, frame_rate=25)
    encoder.to_file(f"../../videolq_videos/{i}.mp4", codec="libx264", crf=0, pixel_format="yuv444p")